In [1]:
import math
import time
import json
import scipy
import pickle
import altair as alt
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats

import matplotlib.pyplot as plt
from numpy import cos, sin, arcsin, sqrt
%matplotlib inline
plt.figure(figsize=(20,5))
PALETTE = ["#FFC30D", "#EF404B"]
PALETTE_ALT = ["#40EFE4", "#EF404B"]
PALETTE_DEG = ["FF"]

<Figure size 1440x360 with 0 Axes>

In [2]:
df = pd.read_pickle('../data/shippify_icd_new.pkl', compression='gzip')
df = df.dropna(subset=['pickup_effective_dt', 'delivery_effective', 'delivery_type'])

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 226828 entries, 0 to 251822
Data columns (total 22 columns):
 #   Column               Non-Null Count   Dtype              
---  ------               --------------   -----              
 0   id                   226828 non-null  object             
 1   route_id             223343 non-null  object             
 2   creation_date        226828 non-null  datetime64[ns, UTC]
 3   delivery_type        226828 non-null  object             
 4   city                 226828 non-null  int8               
 5   cost                 226828 non-null  float64            
 6   total_size           226828 non-null  int64              
 7   distance             226828 non-null  float64            
 8   company_id           226828 non-null  int64              
 9   company_type         226828 non-null  int64              
 10  network_id           226828 non-null  int16              
 11  pickup_dt            226828 non-null  datetime64[ns, UTC]
 12  pi

### Teoria
Tendencia: indica la dirección hacia la cual se dirige la serie de tiempo, característica que lo convierte en el componente más importante. Puede ser creciente, decreciente, constante, lineal, curvilínea, entre otras; se llama también tendencia a largo plazo y se representa con T t .

Ciclo: indica las variaciones que ocurren en una serie de tiempo en períodos más prolongados. Cuando la métrica es en años son variaciones mayores de un año, comúnmente de 2 a 10 años. La serie sube y baja suavemente a manera de onda siguiendo la tendencia. Dicho ciclo puede ser causado por diversos cambios y se representa con C t .

Estacionalidad: indica las variaciones que ocurren a corto plazo en una serie de tiempo con respecto a la línea de tendencia general. Ocurre en períodos fijos como días, semanas, meses, trimestres o años y se representa con E t .

Movimientos irregulares (aleatorios): son oscilaciones de una serie temporal a corto plazo y que se atribuyen a factores imprevisibles o aleatorios. Corresponde al efecto de diversos factores a menudo desconocidos y se representa con A t .

### Pregunta de negocio: Existe tendencia en el aumento de entregas durante el 2do semestre de 2020?

In [4]:
p01 = (pd.to_datetime(df['delivery_effective'])
       .dt.floor('d')
       .value_counts()
       .rename_axis('delivery date')
       .reset_index(name='delivery amount'))
p01['delivery type'] = df['delivery_type']
p01 = p01.dropna()
p01

,delivery date,delivery amount,delivery type
0,2020-06-16 00:00:00+00:00,5038,slot
1,2020-06-17 00:00:00+00:00,4244,flex
2,2020-06-08 00:00:00+00:00,3986,flex
4,2020-06-12 00:00:00+00:00,3841,flex
5,2020-08-12 00:00:00+00:00,3829,flex
...,...,...,...
100,2020-11-10 00:00:00+00:00,1,flex
101,2020-10-27 00:00:00+00:00,1,flex
102,2020-10-13 00:00:00+00:00,1,flex
103,2020-09-16 00:00:00+00:00,1,flex


In [5]:
alt.Chart(p01).mark_line().encode(
    x=alt.X(shorthand='delivery date', type="temporal"),
    y=alt.Y(shorthand='delivery amount', type='quantitative'),
    color=alt.Color(shorthand='delivery type', type="nominal"),
)

alt.Chart(...)

Respuesta: Para esta conclusión debemos tener en cuenta que las entregas se dividen en 2 tipos "flex" y "slot", para el primer tipo vemos que desde Junio hasta septiembre hay estacionalidad en un intervalo de 1 semana. Para el mismo tipo 'slot') vemos que a partir de septiembre hasta diciembre de 2020 no tamos una tendencia a la baja de entregas diarias. Para el tipo slot notamos que desde mediados de Junio existe una clara tendencia a la baja hasta septiembre. Probablemente esta tendencia a la baja se deba a "reglas de negocio" que son desconocidas para nosotros.

#### Pregunta de negocio: Existe estacionalidad con respecto al retraso de entregas durante cada mes del 2do semestre de 2020?

In [6]:
df['delivery_effective_floored'] = pd.to_datetime(df['delivery_effective']).dt.floor('d')

In [7]:
def sumCost(df):
    return np.sum(df['cost'])

In [8]:
groupedDf = df[['delivery_effective_floored','cost']].groupby(
    'delivery_effective_floored').sum().reset_index()

In [9]:
groupedDf.rename(columns={'delivery_effective_floored': 'delivery date', 'cost': 'delivery cost'}, inplace=True)

In [10]:
groupedDf = groupedDf[groupedDf['delivery date'] <= '2020-09-01']

In [11]:
alt.Chart(groupedDf).mark_line().encode(
    x=alt.X(shorthand='delivery date', type="temporal"),
    y=alt.Y(shorthand='delivery cost', type='quantitative'),
)

alt.Chart(...)

### Respuesta: Limitamos la respuesta hasta finales de agosto debido a que desde septiembre no hay la suficiente cantidad de datos para ser analizado. En nuestro análisis encontramos estacionalidad una vez más con respecto a costo diario de las entregas, siempre empieza alto y tiene a caer al final del mes, este ciclo se repite cada semana